<a href="https://colab.research.google.com/github/albo-ai/FastPhotoStyle/blob/master/CapsuleNet_on_Tensorflow_with_Pretrained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [CapsuleNet](https://arxiv.org/pdf/1710.09829v1.pdf)
**Problems with Traditional CNNs**
![alt text](https://cdn-images-1.medium.com/max/800/1*h-Be4Uj-a7N6_Qx-K03KkQ.png)
![alt text](https://cdn-images-1.medium.com/max/800/1*RS9iLmj0r9i9zUeBVzo8Kg.png)

Source: https://hackernoon.com/capsule-networks-are-shaking-up-ai-heres-how-to-use-them-c233a0971952


# New Section

In [0]:
!git clone https://github.com/bourdakos1/capsule-networks.git
!pip install -r capsule-networks/requirements.txt
!touch capsule-networks/__init__.py
!mv capsule-networks capsule
!mv capsule/data/ ./data/
! ls


capsule  data  datalab


The MNIST data set is 60,000 training images. By default the model will be trained for 50 epochs at a batch size of 128.

In [0]:
import os
import tensorflow as tf
from tqdm import tqdm

from capsule.config import cfg
from capsule.utils import load_mnist
from capsule.capsNet import CapsNet

ImportError: ignored

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [0]:
capsNet = CapsNet(is_training=cfg.is_training)

INFO:tensorflow:Seting up the main structure


## CapsuleNet Architecture on the TensorBoard

# New Section

![CapsuleNet Architecture](http://gdurl.com/UtML)


In [0]:
tf.logging.info('Graph loaded')
sv = tf.train.Supervisor(graph=capsNet.graph,
                             logdir=cfg.logdir,
                             save_model_secs=0)

path = cfg.results + '/accuracy.csv'
if not os.path.exists(cfg.results):
    os.mkdir(cfg.results)
elif os.path.exists(path):
    os.remove(path)

fd_results = open(path, 'w')
fd_results.write('step,test_acc\n')

INFO:tensorflow:Graph loaded


In [0]:
with sv.managed_session() as sess:
    num_batch = int(60000 / cfg.batch_size)
    num_test_batch = 10000 // cfg.batch_size
    teX, teY = load_mnist(cfg.dataset, False)
    for epoch in range(cfg.epoch):
        if sv.should_stop():
            break
        for step in tqdm(range(num_batch), total=num_batch, ncols=70, leave=False, unit='b'):
            global_step = sess.run(capsNet.global_step)
            sess.run(capsNet.train_op)

            if step % cfg.train_sum_freq == 0:
                _, summary_str = sess.run([capsNet.train_op, capsNet.train_summary])
                sv.summary_writer.add_summary(summary_str, global_step)

            if (global_step + 1) % cfg.test_sum_freq == 0:
                test_acc = 0
                for i in range(num_test_batch):
                    start = i * cfg.batch_size
                    end = start + cfg.batch_size
                    test_acc += sess.run(capsNet.batch_accuracy, {capsNet.X: teX[start:end], capsNet.labels: teY[start:end]})
                test_acc = test_acc / (cfg.batch_size * num_test_batch)
                fd_results.write(str(global_step + 1) + ',' + str(test_acc) + '\n')
                fd_results.flush()

        if epoch % cfg.save_freq == 0:
            sv.saver.save(sess, cfg.logdir + '/model_epoch_%04d_step_%02d' % (epoch, global_step))
fd_results.close()
tf.logging.info('Training done')


INFO:tensorflow:Starting standard services.
INFO:tensorflow:Starting queue runners.


  0%|                                          | 0/468 [00:00<?, ?b/s]

INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:Recording summary at step 0.


  0%|▏                               | 2/468 [01:50<7:07:41, 55.07s/b]

INFO:tensorflow:Recording summary at step 3.
INFO:tensorflow:global_step/sec: 0.0206392


...

 59%|████████████████▍           | 274/468 [2:43:36<1:55:50, 35.83s/b]

INFO:tensorflow:Recording summary at step 280.


 59%|████████████████▋           | 278/468 [2:45:54<1:53:23, 35.81s/b]

INFO:tensorflow:Recording summary at step 284.


 60%|████████████████▋           | 279/468 [2:46:39<1:52:53, 35.84s/b]

# Pre-trained model
So this took me approx 6 hrs to train it on **NVIDIA TitanXp** GPU.

You can download the pre-trained model for MNIST dataset here
https://goo.gl/DN7SS3





Thanks,

Rahul Kumar

www.hellorahulk.com
